# **Monotonic constraints experiments**

# **Set-up**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"
pio.templates.default = "simple_white"
warnings.filterwarnings('ignore')

# Import ML
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef
from catboost import CatBoostClassifier, Pool


In [38]:
train_uncleaned = pd.read_csv('Data/train.csv') # includes the original data 
test_uncleaned = pd.read_csv('Data/test.csv') # includes the original data 
train = pd.read_csv('Data/train_cleaned.csv') # includes the original data 
test = pd.read_csv('Data/test_cleaned.csv')
original = pd.read_csv('Data/original_cleaned.csv')
train.drop(columns=['cgpa_bucket'], inplace=True)
train.head()

,is_female,age,city,is_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,suicidal_thoughts,work_study_hours,financial_stress,family_history,depression,is_original
0,1,49,Ludhiana,0,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,0,1.0,2.0,0,0,0
1,0,26,Varanasi,0,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,1,7.0,3.0,0,1,0
2,0,33,Visakhapatnam,1,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,1,3.0,1.0,0,1,0
3,0,22,Mumbai,0,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,1,10.0,1.0,1,1,0
4,1,30,Kanpur,0,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,1,9.0,4.0,1,0,0


# **Relationship with the target**

In [3]:
# Lets see the relationship between the features and the target. 
# Separate numerical and categorical features
num_features = train.select_dtypes(include=['int32', 'float32']).columns.tolist()
cat_features = train.select_dtypes(include=['object', 'string']).columns.tolist()

# Remove target and is_original from features
num_features = [f for f in num_features if f not in ['depression', 'is_original']]
cat_features = [f for f in cat_features if f not in ['depression', 'is_original']]

In [18]:
# Remove target and is_original
features = [f for f in train.columns if f not in ['depression', 'is_original']]

# Calculate depression rate for each feature value
fig = make_subplots(rows=len(features)//3 + 1, cols=3, 
                    subplot_titles=features)

for i, feat in enumerate(features):
    # Calculate proportions
    props = (train.fillna({feat: -1 if original[feat].dtype.kind in 'biufc' else 'NA'})
            .groupby(feat)['depression']
            .agg(['count', 'mean'])
            .assign(pct=lambda x: x['mean'] * 100)
            .round(2)
            .sort_values('mean', ascending=False))
    
    # Add count as hover text
    hover_text = [f"Count: {c}" for c in props['count']]
    
    fig.add_trace(
        go.Bar(x=props.index, 
               y=props['pct'], 
               text=props['pct'].apply(lambda x: f"{x:.1f}%"),
               hovertext=hover_text,
               name=feat),
        row=i//3 + 1, col=i%3 + 1
    )

fig.update_layout(height=300*((len(features)//3) + 1), 
                 width=1000, 
                 showlegend=False,
                 title_text="Depression Rate by Feature Values")
fig.show()

The results are very interesting:

- Beign younger (age < 20), student increases significantly the probability of having depression. Students in particularly is a defining factor in this dataset.
- As expected, ordered features correlate with depression:
    - `academic_pressure`, `work_pressure`, `financial_stress`, `work_study_hours` as it increases, the probability of having depression increases.
    - `study_satisfaction`, `job_satisfaction` as it decreases, the probability of having depression increases.
- Having bad `dietary_habits` and `sleep_duration` decreases the probability of having depression.
- `Gender` and `family_history` do not show a strong correlation with depression. However, they may have interaction with other features.
- `Graphic Designer` is the top profession with high proportion of depression.
- `Hyderabad` is the top city with high proportion of depression. This could be biased because there are more students living there and also is an importante tech hub in India where there is high academic pressure.
- Having `suicidal_thoughts` is not a good sign, 31% of the people with suicidal thoughts have depression, whereas only 5% of the people without suicidal thoughts have depression.
- `Class 12` in India is a crucial year in a student's life because it is the year when students take very important exams that may determine their future. This may explain why there is a high proportion of depression in this group.
- **Missing Values** usually have a high proportion of depression in certain features like `work_pressure`, `financial_stress` because all of the NAs are practically students. Whereas, missing values in student related features like `academic_pressure`, `study_satisfaction` have low proportions of depression because there are not students. 

# **Encoding ordinal features**

In [51]:
# academic pressure
academic_pressure_mapping = {
    -1: 0,  # lowest depression rate
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6
}

# work pressure
work_pressure_mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    -1: 6
}

# study satisfaction
study_satisfaction_mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    -1: 6
}

# Job satisfaction
job_satisfaction_mapping = {
    -1: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5
}

# Sleep duration (ordering based on depression rates)
sleep_mapping = {
    'Missing': 2,                  # middle depression rate
    'Less than 5 hours': 0,   # highest depression rate
    '5-6 hours': 2,
    '7-8 hours': 1,
    'More than 8 hours': 3    # lowest depression rate
}


# Dietary habits (NA -> Healthy -> Moderate -> Unhealthy)
dietary_mapping = {
    'Missing': 0,        # highest depression rate
    'Unhealthy': 1,
    'Moderate': 2,
    'Healthy': 3    # lowest depression rate
}

# Work study hours
work_study_hours_mapping = {
    0: 1,
    1: 1,
    2: 2,
    3: 3,
    4: 3,
    5: 3,
    6: 5,
    7: 6,
    8: 7,
    9: 4,
    10: 8,
    11: 7,
    12: 8
}

# Financial stress
financial_stress_mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    -1: 5,
    5: 6
}

In [54]:
# Function to apply custom mapping with missing values
def apply_custom_mapping(series, mapping):
    return series.map(mapping)

# Fill NaN in categorical features with 'Missing'
train_prep = train.copy()
test_prep = test.copy()

for feat in cat_features:
    train_prep[feat] = train_prep[feat].fillna('Missing')
    test_prep[feat] = test_prep[feat].fillna('Missing')

ordered_features = ['academic_pressure', 'work_pressure', 'study_satisfaction', 'job_satisfaction', 'financial_stress']
for feat in ordered_features:
    # Missing values in financial stress should be -1 for mapping to work
    train_prep[feat] = train_prep[feat].fillna(-1)
    test_prep[feat] = test_prep[feat].fillna(-1)

# Apply the mappings for train
train_prep['work_study_hours'] = apply_custom_mapping(train_prep['work_study_hours'], work_study_hours_mapping)
train_prep['financial_stress'] = apply_custom_mapping(train_prep['financial_stress'], financial_stress_mapping)
train_prep['academic_pressure'] = apply_custom_mapping(train_prep['academic_pressure'], academic_pressure_mapping)
train_prep['dietary_habits'] = apply_custom_mapping(train_prep['dietary_habits'], dietary_mapping)
train_prep['sleep_duration'] = apply_custom_mapping(train_prep['sleep_duration'], sleep_mapping)
train_prep['work_pressure'] = apply_custom_mapping(train_prep['work_pressure'], work_pressure_mapping)
train_prep['job_satisfaction'] = apply_custom_mapping(train_prep['job_satisfaction'], job_satisfaction_mapping)
train_prep['study_satisfaction'] = apply_custom_mapping(train_prep['study_satisfaction'], study_satisfaction_mapping)

# Apply the mappings for test 
test_prep['academic_pressure'] = apply_custom_mapping(test_prep['academic_pressure'], academic_pressure_mapping)
test_prep['dietary_habits'] = apply_custom_mapping(test_prep['dietary_habits'], dietary_mapping)
test_prep['sleep_duration'] = apply_custom_mapping(test_prep['sleep_duration'], sleep_mapping)
test_prep['work_study_hours'] = apply_custom_mapping(test_prep['work_study_hours'], work_study_hours_mapping)
test_prep['financial_stress'] = apply_custom_mapping(test_prep['financial_stress'], financial_stress_mapping)
test_prep['work_pressure'] = apply_custom_mapping(test_prep['work_pressure'], work_pressure_mapping)
test_prep['job_satisfaction'] = apply_custom_mapping(test_prep['job_satisfaction'], job_satisfaction_mapping)
test_prep['study_satisfaction'] = apply_custom_mapping(test_prep['study_satisfaction'], study_satisfaction_mapping)

In [55]:
# Remove target and is_original
features = [f for f in train_prep.columns if f not in ['depression', 'is_original']]

# Calculate depression rate for each feature value
fig = make_subplots(rows=len(features)//3 + 1, cols=3, 
                    subplot_titles=features)

for i, feat in enumerate(features):
    # Calculate proportions
    props = (train_prep
            .groupby(feat)['depression']
            .agg(['count', 'mean'])
            .assign(pct=lambda x: x['mean'] * 100)
            .round(2)
            .sort_values('mean', ascending=False))
    
    # Add count as hover text
    hover_text = [f"Count: {c}" for c in props['count']]
    
    fig.add_trace(
        go.Bar(x=props.index, 
               y=props['pct'], 
               text=props['pct'].apply(lambda x: f"{x:.1f}%"),
               hovertext=hover_text,
               name=feat),
        row=i//3 + 1, col=i%3 + 1
    )

fig.update_layout(height=300*((len(features)//3) + 1), 
                 width=1000, 
                 showlegend=False,
                 title_text="Depression Rate by Feature Values")
fig.show()

# **Testing monotonic constraints with catboost**


In [ ]:
# Split train/test
X = train_prep[features]
y = train_prep['depression']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)